In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import re

df_true = pd.read_csv('True.csv')
df_fake = pd.read_csv('Fake.csv')

In [ ]:
print("Total data True = ", len(df_true))
print('total data fake = ', len(df_fake))
df_true

Total data True =  21417
total data fake =  23481


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [ ]:
df_true.drop(columns=(['subject', 'date']), inplace=True)
df_fake.drop(columns=(['subject', 'date']), inplace=True)

In [ ]:
df_true['label'] = 0
df_fake['label'] = 1

In [ ]:
df_true.head(1)

,title,text,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,0


In [ ]:
df_fake.head(1)

,title,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,1


In [ ]:
pattern = "^[A-Z/]+ \(Reuters\) - " 

In [ ]:
sum([1 for text in df_true["text"] if not re.match(pattern, text)])

5675

In [ ]:
def clean(text: str) -> str:
    return re.sub(pattern, "", text)

In [ ]:
df_true['text'] = df_true['text'].apply(clean)
df_fake['text'] = df_fake['text'].apply(clean)
df

,title,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,1
...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,NATO allies on Tuesday welcomed President Dona...,0
21413,LexisNexis withdrew two products from Chinese ...,"LexisNexis, a provider of legal, regulatory an...",0
21414,Minsk cultural hub becomes haven from authorities,In the shadow of disused Soviet-era factories ...,0
21415,Vatican upbeat on possibility of Pope Francis ...,Vatican Secretary of State Cardinal Pietro Par...,0


In [ ]:
df = pd.concat([df_fake, df_true])
df

,title,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,1
...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,NATO allies on Tuesday welcomed President Dona...,0
21413,LexisNexis withdrew two products from Chinese ...,"LexisNexis, a provider of legal, regulatory an...",0
21414,Minsk cultural hub becomes haven from authorities,In the shadow of disused Soviet-era factories ...,0
21415,Vatican upbeat on possibility of Pope Francis ...,Vatican Secretary of State Cardinal Pietro Par...,0


In [ ]:
x_title = df[['title','text']]
y_label = df['label']
features_dict = {name: np.array(value) for name, value in x_title.items()}

In [ ]:
from sklearn.model_selection import train_test_split

x_title = df[['title','text']]
y_label = df['label']
x_train, x_test, y_train, y_test = train_test_split(x_title, y_label, test_size=0.2)

In [ ]:
def get_output_sequence_length(column):
    lengths = [len(x.split()) for x in column]
    return max(lengths)

In [ ]:
inputs = {}
for name, column in x_title.items():    
    inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=tf.string)
    
outputs = []
for name, input in inputs.items():
    output_sequence_length = get_output_sequence_length(x_title[name])
    text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=output_sequence_length)
    text_vectorizer.adapt(x_title[name])
    
    x = text_vectorizer(input)
    outputs.append(x)
    

In [ ]:
!wget --no-check-certificate \
     http://nlp.stanford.edu/data/glove.6B.zip \
     -O /tmp/glove.6B.zip

--2022-11-25 09:07:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-25 09:07:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-25 09:07:46--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘/tmp/glove.6B.zip’


In [ ]:
import zipfile, os
with zipfile.ZipFile("/tmp/glove.6B.zip", "r") as zip_ref:
    zip_ref.extractall("/tmp/glove")

embeddings_index = {}
f = open("/tmp/glove/glove.6B.100d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [ ]:
embedding_matrix = np.zeros((text_vectorizer.vocabulary_size(), 100))
for i, word in enumerate(text_vectorizer.get_vocabulary()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
# Tokenizer mengubah kalimat kedalam numeric
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(x_train)
tokenizer.fit_on_texts(x_test)

sekuens_latih = tokenizer.texts_to_sequences(x_train)
sekuens_test = tokenizer.texts_to_sequences(x_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=text_vectorizer.vocabulary_size(), output_dim=100, weights=[embedding_matrix], input_length=100, trainable=False),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dense(64, activation='relu'),
    # tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=10)]

In [ ]:
model.fit(padded_train,
          y_label, 
          epochs=30, 
          validation_split=0.25,
          batch_size=512,
          verbose=2,
          )

Epoch 1/30


1/1 - 2s - loss: 0.6608 - accuracy: 1.0000 - val_loss: 0.5954 - val_accuracy: 1.0000 - 2s/epoch - 2s/step
Epoch 2/30
1/1 - 0s - loss: 0.6086 - accuracy: 1.0000 - val_loss: 0.5661 - val_accuracy: 1.0000 - 28ms/epoch - 28ms/step
Epoch 3/30
1/1 - 0s - loss: 0.5591 - accuracy: 1.0000 - val_loss: 0.5359 - val_accuracy: 1.0000 - 27ms/epoch - 27ms/step
Epoch 4/30
1/1 - 0s - loss: 0.5144 - accuracy: 1.0000 - val_loss: 0.5053 - val_accuracy: 1.0000 - 24ms/epoch - 24ms/step
Epoch 5/30
1/1 - 0s - loss: 0.4724 - accuracy: 1.0000 - val_loss: 0.4750 - val_accuracy: 1.0000 - 24ms/epoch - 24ms/step
Epoch 6/30
1/1 - 0s - loss: 0.4313 - accuracy: 1.0000 - val_loss: 0.4437 - val_accuracy: 1.0000 - 23ms/epoch - 23ms/step
Epoch 7/30
1/1 - 0s - loss: 0.3919 - accuracy: 1.0000 - val_loss: 0.4124 - val_accuracy: 1.0000 - 27ms/epoch - 27ms/step
Epoch 8/30
1/1 - 0s - loss: 0.3538 - accuracy: 1.0000 - val_loss: 0.3816 - val_accuracy: 1.0000 - 23ms/epoch - 23ms/step
Epoch 9/30
1/1 - 0s - loss: 0.3161 - accuracy: 